In [106]:
import numpy as np
from bs4 import BeautifulSoup

In [168]:
def dot(p1, p2):
    """
    Dot product for 4-vectors of the form (E, px, py, pz).
    Returns: E1*E2 - p1*p2  
    """
    return p1[0] * p2[0] - p1[1] * p2[1] - p1[2] * p2[2] - p1[3] * p2[3]

In [169]:
class Process:
    """
    Class process that takes a text file in one of the supported formats and recognise events and particles.
    Return: object of the class.
    """
    
    supported_file_formats = ['lhe']
    
    def __init__(self, txt_file):
        
        file_format = txt_file.split('.')[-1]
        
        if file_format not in Process.supported_file_formats:
            print("File format %s not recognised." % file_format)
        else:
            if file_format == 'lhe':
                self.events = self.read_lhe(txt_file)
                
            self._num_events = len(self.events)
            
            self._cross_section = sum([event.weight for event in self.events])
    
    def read_lhe(self, txt_file):
        
        with open(txt_file, 'r') as f:
            soup = BeautifulSoup(f, "lxml")

            events = []

            for event in soup.find_all('event'):
                events.append(Event(event.string.strip('\n').split('\n'), 'lhe'))
        
        return events
    
    @property
    def num_events(self):
        return self._num_events
    
    @property
    def cross_section(self):
        return self._cross_section
    
    def __str__(self):
        return "Number of events: %i\nCross section: %f\n" % (self.num_events, self.cross_section) +\
               ''.join([event.__str__() for event in self.events])
            
        

In [174]:
class Event:
    """
    Event class, reads the event provided as a list of strings in a specified format,
    extracts the weight and defines the various particles.
    Return: object of the class.
    """
    
    def __init__(self, event, event_type):
        
        if event_type == 'lhe':
            
            general_info = event[0].strip().split()
            self._tot_particles = int(general_info[0])
            self._weight = float(general_info[2])
            
            self.particles = []
            
            for line in event[1:]:
                line = line.strip().split()
                
                if line[0].lstrip('-').isdigit():
                    self.particles.append(Particle(line, 'lhe'))
            
            self.initial_states = [particle for particle in self.particles if particle.status == -1]
            self.final_states = [particle for particle in self.particles if particle.status == 1]
    
    @property
    def weight(self):
        return self._weight
    
    @property
    def tot_particles(self):
        return self._tot_particles
    
    def __str__(self):
        return "\nEvent weight: %f\nTotal number of particles: %i\n" % (self.weight, self.tot_particles) +\
               ''.join([particle.__str__() for particle in self.particles])

In [171]:
class Particle:
    """
    Particle class, reads a string in a specific format describing a particle.
    Return: object of the class.
    """
    
    def __init__(self, particle_info, particle_type):
        
        if particle_type == 'lhe':
            self._pdg = int(particle_info[0])
            self._status = int(particle_info[1])
            self._px = float(particle_info[6])
            self._py = float(particle_info[7])
            self._pz = float(particle_info[8])
            self._E = float(particle_info[9])
            self._M = float(particle_info[10])
            self._ctau = float(particle_info[11])
    
    @property
    def pdg(self):
        return self._pdg

    @property
    def status(self):
        return self._status

    @property
    def px(self):
        return self._px

    @property
    def py(self):
        return self._py

    @property
    def pz(self):
        return self._pz

    @property
    def E(self):
        return self._E

    @property
    def M(self):
        return self._M

    @property
    def ctau(self):
        return self._ctau
    
    @property
    def mom(self):
        return np.array([self._E, self._px, self._py, self._pz])
    
    @property
    def p3(self):
        return np.sqrt(self._px ** 2 + self._py ** 2 + self._pz ** 2)
    
    @property
    def pT(self):
        return np.sqrt(self._px**2 + self._py**2)
    
    @property
    def y(self):
        return 0.5 * np.log((self._E + self._pz)/(self._E - self._pz))
    
    @property
    def eta(self):
        return 0.5 * np.log((self.p3 + self._pz)/(self.p3 - self._pz))
    
    @property
    def ET(self):
        return np.sqrt(self._M ** 2 + self.pT ** 2)
    
    @property
    def beta(self):
        return self.p3 / self._E
    
    @property
    def gamma(self):
        return 1 / np.sqrt(1 - self.beta ** 2)
    
    @property
    def theta(self):
        return np.arctan2(self.pT, self._pz)
    
    @property
    def phi(self):
        return np.arctan2(self._py, self._px)
    
    def __str__(self):
        return "\nParticle information:\n" +\
               "pdg: %i\n" % self.pdg +\
               "status: %i\n" % self.status +\
               "px: %f\n" % self.px +\
               "py: %f\n" % self.py +\
               "pz: %f\n" % self.pz +\
               "E: %f\n" % self.E +\
               "M: %f\n" % self.M +\
               "ctau: %f\n" % self.ctau

In [172]:
# prova = Process("./data/pythia8_BasicReco.lhe")
prova = Process("./data/unweighted_events.lhe")

In [162]:
prova.cross_section

0.5825999999999999

In [152]:
print(prova.events[0].particles[0])


Particle information:
pdg: 2
status: 3
px: 0.000000
py: 0.000000
pz: 2048.476562
E: 2048.476562
M: 0.000000
ctau: 0.000000



In [153]:
print(prova.events[0])


Event weight: 0.000058
Total number of particles: 18

Particle information:
pdg: 2
status: 3
px: 0.000000
py: 0.000000
pz: 2048.476562
E: 2048.476562
M: 0.000000
ctau: 0.000000

Particle information:
pdg: -2
status: 3
px: 0.000000
py: 0.000000
pz: -651.298706
E: 651.298706
M: 0.000000
ctau: 0.000000

Particle information:
pdg: 4000002
status: 3
px: 501.519836
py: 283.560761
pz: 753.914673
E: 1378.519531
M: 1000.000000
ctau: 0.000000

Particle information:
pdg: -4000002
status: 3
px: -501.519836
py: -283.560761
pz: 643.263062
E: 1321.255615
M: 1000.000000
ctau: 0.000000

Particle information:
pdg: 51
status: 3
px: -28.984419
py: 249.934799
pz: -200.622803
E: 321.958069
M: 10.000000
ctau: 0.000000

Particle information:
pdg: 2
status: 3
px: 628.244934
py: 142.605911
pz: 857.580688
E: 1072.600952
M: 0.330000
ctau: 0.000000

Particle information:
pdg: -51
status: 3
px: -68.141975
py: 334.834717
pz: -135.817245
E: 367.836853
M: 10.000000
ctau: 0.000000

Particle information:
pdg: -2
status

In [154]:
print(prova)

Number of events: 100
Cross section: 0.005826

Event weight: 0.000058
Total number of particles: 18

Particle information:
pdg: 2
status: 3
px: 0.000000
py: 0.000000
pz: 2048.476562
E: 2048.476562
M: 0.000000
ctau: 0.000000

Particle information:
pdg: -2
status: 3
px: 0.000000
py: 0.000000
pz: -651.298706
E: 651.298706
M: 0.000000
ctau: 0.000000

Particle information:
pdg: 4000002
status: 3
px: 501.519836
py: 283.560761
pz: 753.914673
E: 1378.519531
M: 1000.000000
ctau: 0.000000

Particle information:
pdg: -4000002
status: 3
px: -501.519836
py: -283.560761
pz: 643.263062
E: 1321.255615
M: 1000.000000
ctau: 0.000000

Particle information:
pdg: 51
status: 3
px: -28.984419
py: 249.934799
pz: -200.622803
E: 321.958069
M: 10.000000
ctau: 0.000000

Particle information:
pdg: 2
status: 3
px: 628.244934
py: 142.605911
pz: 857.580688
E: 1072.600952
M: 0.330000
ctau: 0.000000

Particle information:
pdg: -51
status: 3
px: -68.141975
py: 334.834717
pz: -135.817245
E: 367.836853
M: 10.000000
ctau: 

In [155]:
for particle in prova.events[0].final_states:
    print(particle)


Particle information:
pdg: 21
status: 1
px: -2.081032
py: 4.945336
pz: -5.319717
E: 7.766984
M: 1.799892
ctau: 0.000000


Particle information:
pdg: 21
status: 1
px: -6.650265
py: -3.215879
pz: -1703.544556
E: 1703.561645
M: 1.926397
ctau: 0.000000


Particle information:
pdg: 21
status: 1
px: -5.179947
py: -16.288071
pz: 85.678352
E: 87.402451
M: 2.504864
ctau: 0.000000


Particle information:
pdg: 21
status: 1
px: 22.959549
py: 32.941967
pz: -74.623894
E: 85.224052
M: 9.060830
ctau: 0.000000


Particle information:
pdg: 21
status: 1
px: -50.860416
py: -23.175804
pz: 6.743088
E: 56.745380
M: 7.118220
ctau: 0.000000


Particle information:
pdg: 21
status: 1
px: -149.063141
py: -28.733624
pz: 182.787613
E: 238.270050
M: 17.772738
ctau: 0.000000


Particle information:
pdg: 21
status: 1
px: -62.826122
py: -208.823013
pz: -185.526459
E: 286.786377
M: 16.497871
ctau: 0.000000


Particle information:
pdg: 21
status: 1
px: -279.838318
py: -470.161713
pz: 483.840240
E: 731.757935
M: 44.79729